In [ ]:
import gradio as gr
import openai
import json

openai.api_key = 'your-openai-api-key'

book_files = {
    "book1": "/content/book1.txt",
    "book2": "/content/book2.txt",
    "book3": "/content/book3.txt"
}

hierarchical_indices = {}
for book_name in book_files.keys():
    with open(f'hierarchical_index_{book_name}.json', 'r', encoding='utf-8') as f:
        hierarchical_indices[book_name] = json.load(f)

# Function to extract content from the hierarchical tree
def extract_content(node):
    if node is None:
        return ''

    content = node.get('content', '')

    for child in node.get('children', []):
        child_content = extract_content(child)
        if child_content:
            content += ' ' + child_content

    return content

# Extracting content from each book
book_contents = {}
for book_name, index in hierarchical_indices.items():
    book_contents[book_name] = extract_content(index)

# Function to answer questions using OpenAI GPT-4
def answer_question(book_choice, question):
    if book_choice not in book_contents:
        return "Invalid book choice"

    context = book_contents[book_choice]

    if not context:
        return "The context for the question is empty."

    # OpenAI GPT-3.5 to generate a detailed answer
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct	",
        prompt=f"Context: {context}\n\nQuestion: {question}\n\nAnswer in detail:",
        max_tokens=300
    )
    answer = response.choices[0].text.strip()
    return answer

def main():
    with gr.Blocks() as demo:
        gr.Markdown("## Question Answering with Hierarchical Index")

        book_choice = gr.Dropdown(choices=list(book_files.keys()), label="Choose a Book")
        question = gr.Textbox(label="Enter your question")
        output = gr.Textbox(label="Answer", interactive=False)

        question.submit(fn=answer_question, inputs=[book_choice, question], outputs=output)

    demo.launch()

main()